<a href="https://colab.research.google.com/github/DaraRahma536/TensorFlow-in-Action/blob/main/Chapter_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 6: Teaching Machines to See - Image Classification with CNNs**

# **1. Exploratory Data Analysis (EDA) untuk Data Gambar**
---
### **A. Tujuan EDA**
* Memahami karakteristik data
* Mengidentifikasi masalah: class imbalance, data korup, missing values, outlier
* Menjamin kualitas data sebelum training model

### **B. Dataset: Tiny ImageNet-200**
* Subset dari ImageNet dengan 200 kelas
* 100,000 gambar training (500 per kelas)
* 10,000 gambar validation
* Format folder terstruktur:
```
train/
  ├── n01443537/ (wnid)
  │   ├── image1.JPEG
  │   └── ...
  └── ...
```
### **C. Analisis yang Dilakukan**
* Memahami Kelas: Mapping wnid (WordNet ID) ke nama kelas melalui wnids.txt dan words.txt
* Class Balance: Memverifikasi setiap kelas memiliki 500 gambar
* Statistik Gambar: Ukuran gambar seragam 64x64 pixels
* Visualisasi: Contoh gambar menunjukkan variasi konteks dan kondisi nyata

**Insight Penting:** Data real-world berisik - objek tidak selalu jelas, dengan latar belakang beragam, dan terkadang terhalang.

# **2. Membuat Data Pipeline dengan Keras**
---
### **A. ImageDataGenerator**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator(
    samplewise_center=True,  # Normalisasi per gambar
    validation_split=0.1      # Split 90% train, 10% validation
)

### **B. Data Generators**
* flow_from_directory(): Untuk data terstruktur dalam folder
* flow_from_dataframe(): Untuk data dengan label di file terpisah
* flow(): Untuk data dalam array NumPy

### **C. Implementasi Pipeline**

In [ ]:
# Training generator
train_gen = image_gen.flow_from_directory(
    directory='data/tiny-imagenet-200/train',
    target_size=(56, 56),  # Resize untuk Inception Net
    batch_size=128,
    class_mode='categorical',
    subset='training'
)

# Validation generator (dari training split)
valid_gen = image_gen.flow_from_directory(
    ...,
    subset='validation'
)

# Test generator (dari folder val)
test_df = pd.read_csv('val_annotations.txt', sep='\t')
test_gen = image_gen.flow_from_dataframe(
    dataframe=test_df,
    directory='data/tiny-imagenet-200/val/images',
    shuffle=False  # Penting untuk evaluasi konsisten
)

### **D. Modifikasi untuk Multi-Output Model**
Inception Net memiliki 3 output layer (1 utama + 2 auxiliary), sehingga label perlu direplikasi:

In [ ]:
def data_gen_aux(gen):
    for x, y in gen:
        yield x, (y, y, y)  # Satu label untuk tiga output

# **3. Inception Net: Model State-of-the-Art**
---
### **A. Motivasi**
* Masalah: Model CNN dalam cenderung overfit dan membutuhkan memori besar
* Solusi: Sparsity melalui arsitektur Inception Block

### **B. Arsitektur Inception Net v1**
**1. Stem**
* Serangkaian layer konvolusi dan pooling awal
* Menggunakan Local Response Normalization (LRN) (teknik normalisasi awal)
* Output: Feature maps dengan dimensi tertentu

**2. Inception Block (Konsep Inti)**
* Paralel convolution layers dengan kernel size berbeda (1x1, 3x3, 5x5)
* Concatenation output semua parallel streams
* Manfaat:
-&nbsp;Sparsity: Koneksi lebih jarang daripada convolution tunggal  
-&nbsp;Parameter Efficiency: Lebih sedikit parameter dengan kemampuan representasi sama  
-&nbsp;Multi-scale features: Menangkap pola pada skala berbeda  
* Analogi: Alih-alih filter 5x5 tunggal, gunakan kombinasi 1x1, 3x3, dan 5x5 yang lebih efisien.  

**3. 1×1 Convolutions sebagai Dimensionality Reduction**
* Mengurangi channel depth sebelum convolution mahal (3x3, 5x5)
* Contoh: Input 256 channel → 1x1 conv → 64 channel → 3x3 conv
* Pengurangan parameter signifikan

**4. Auxiliary Output Layers**
* Dua output tambahan di tengah jaringan
* Fungsi: Stabilisasi training deep network
* Membantu gradient flow ke layer awal

### **C. Implementasi Inception Block**

In [ ]:
def inception(inp, n_filters):
    # Stream 1: 1x1 convolution
    out1 = Conv2D(n_filters[0][0], (1,1), activation='relu', padding='same')(inp)

    # Stream 2: 1x1 → 3x3
    out2_1 = Conv2D(n_filters[1][0], (1,1), activation='relu', padding='same')(inp)
    out2_2 = Conv2D(n_filters[1][1], (3,3), activation='relu', padding='same')(out2_1)

    # Stream 3: 1x1 → 5x5
    out3_1 = Conv2D(n_filters[2][0], (1,1), activation='relu', padding='same')(inp)
    out3_2 = Conv2D(n_filters[2][1], (5,5), activation='relu', padding='same')(out3_1)

    # Stream 4: 3x3 pool → 1x1
    out4_1 = MaxPool2D((3,3), strides=(1,1), padding='same')(inp)
    out4_2 = Conv2D(n_filters[3][0], (1,1), activation='relu', padding='same')(out4_1)

    # Concatenate semua stream
    out = Concatenate(axis=-1)([out1, out2_2, out3_2, out4_2])
    return out

### **D. Model Lengkap Inception v1**

In [ ]:
def inception_v1():
    inp = Input(shape=(56, 56, 3))

    # Stem
    stem_out = stem(inp)

    # Inception blocks (9 blocks sesuai paper)
    inc_3a = inception(stem_out, [(64,), (96,128), (16,32), (32,)])
    inc_3b = inception(inc_3a, [(128,), (128,192), (32,96), (64,)])
    # ... (7 blocks lainnya)

    # Auxiliary outputs
    aux_out1 = aux_out(inc_4a, name='aux1')
    aux_out2 = aux_out(inc_4d, name='aux2')

    # Final layers
    avgpool = AvgPool2D((7,7), strides=(1,1), padding='valid')(inc_5b)
    flat = Flatten()(avgpool)
    out_main = Dense(200, activation='softmax', name='final')(flat)

    # Multi-output model
    model = Model(inputs=inp, outputs=[out_main, aux_out1, aux_out2])
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# **4. Training dan Evaluasi**
---
### **A. Training Process**

In [ ]:
model.fit(
    x=train_gen_aux,
    validation_data=valid_gen_aux,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=50,
    callbacks=[CSVLogger('training_log.csv')]
)

### **B. Hasil Training**
* Training accuracy: ~94% (sangat tinggi)
* Validation/Test accuracy: ~27-30% (rendah)
* Indikasi kuat overfitting

### **C. Analisis Overfitting**
Penyebab:
* Model terlalu kompleks untuk dataset
* Kurang regularisasi
* Tidak menggunakan pretrained weights

Analogi: Siswa yang menghafal jawaban (overfit) vs memahami konsep (generalize).

# **5. Evolusi Inception Models**
---
### **A. Inception v2**
* Factorization: Ganti 5x5 convolution dengan dua 3x3 convolution
* Parameter reduction: 28% lebih sedikit parameter
* Ganti 3x3 dengan 1x3 + 3x1 untuk efisiensi lebih

### **B. Inception v3**
* Tambah Batch Normalization
* RMSProp optimizer dengan learning rate decay
* Label smoothing regularization

### **C. Inception v4 & Inception-ResNet**
* Simplifikasi arsitektur
* Residual connections (skip connections)
* Kombinasi Inception blocks dengan ResNet